### GCP Setup

In [ ]:
# Install/upgrade gcloud & Docker
gcloud --version
docker --version

# Log in to Google Cloud & set project/region
PROJECT_ID=debaagent-e2d1c
REGION=europe-west1
gcloud auth login
gcloud config set project $PROJECT_ID
gcloud config set run/region $REGION

# Enable APIs (idempotent)
gcloud services enable run.googleapis.com\
 artifactregistry.googleapis.com\
  secretmanager.googleapis.com\
   firestore.googleapis.com

### Service Account and IAM

In [ ]:
SA_NAME=debaagent-app
SA_EMAIL="$SA_NAME@$PROJECT_ID.iam.gserviceaccount.com"

# Create SA
gcloud iam service-accounts create $SA_NAME display-name "Deba Agent SA" 2>/dev/null || true #ignores errors and keeps running if SA already exists

#Firestore and Secret Manager Access for SA Account
gcloud projects add-iam-policy-binding $PROJECT_ID --member="serviceAccount:$SA_EMAIL" --role="roles/datastore.user"
gcloud projects add-iam-policy-binding $PROJECT_ID --member="serviceAccount:$SA_EMAIL" --role="roles/secretmanager.secretAccessor"

### Secrets

In [ ]:
echo -n "sk-proj-PUT_IN_KEY" | gcloud secrets create OPENAI_API_KEY --data-file=- 2>/dev/null \
|| echo -n "sk-proj-PUT_IN_KEY" | gcloud secrets versions add OPENAI_API_KEY --data-file=-

#This structure attempts to create the API-key, and ignores errors if already existing. Then the second line proceesds to update the API key in that case

#Repeat this for other Keys

### Dockerfile (+ dockerignore)

FROM python:3.11-slim
ENV PYTHONUNBUFFERED=1 PIP_NO_CACHE_DIR=1
WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .
EXPOSE 8080
CMD ["sh","-c","uvicorn deba_app:app --host 0.0.0.0 --port ${PORT:-8080}"]

### Build and push Dockerfile

In [ ]:
REPO=debaagent
IMAGE_NAME=debaagent
AR_IMAGE="$REGION-docker.pkg.dev/$PROJECT_ID/$REPO/$IMAGE_NAME:latest"

# create repo
gcloud artifacts repositories create $REPO\
  --repository-format=docker \
  --location=$REGION \
  --description="debaagent images" 2>/dev/null || true

#auth Docker
gcloud auth configure-docker $REGION-docker.pkg.dev

#buildx (because of silicon) with cache
AR_CACHE="$REGION-docker.pkg.dev/$PROJECT_ID/$REPO/$IMAGE_Name:cache"
docker buildx create --use 2>/dev/null || true  #ensures a buildx builder instance exists and is set as a default
docker buildx build \
  --platform linux/amd64 \
  -t "$AR_IMAGE" \
  --cache-from type=registry,ref="$AR_CACHE" \
  --cache-to type=registry,ref="$AR_CACHE",mode=max \
  --push .

  # Above invokes docker buildx (supports platform and cache)
  # platform: architecture for which image is built (our case: we build on silicon for cloud servers)
  # -t (tag) -> tags the image with the artifact registry path
  # cache-from takes what is already cached
  # cache-to pushes layers to artifact registry for reuse
  # push . pushes to artifact registry

# if you want to push with nothing else involved you do
# docker buildx build -t "$AR_IMG" --push .


### Deploy to Cloud

In [ ]:
SVC=debaagent
SA_EMAIL="$SA_NAME@$PROJECT_ID.iam.gserviceaccount.com"

gcloud run deploy $SVC \
  --image "$AR_IMAGE" \
  --region "$REGION" \
  --allow-unauthenticated \
  --set-env-vars PROJECT_ID="$PROJECT_ID" \
  --update-secrets OPENAI_API_KEY=OPENAI_API_KEY:latest,DB_CLIENT_ID=DB_CLIENT_ID:latest,DB_CLIENT_API_KEY=DB_CLIENT_API_KEY:latest
--

### get URL

In [ ]:
SERVICE_URL=$(glcoud run services describe $SVC --region $REGION --format="value(status.url)")
echo "$SERVICE_URL"